In [ ]:
import folium
import polars as pl
import geopandas as gpd

import fryer.data
import fryer.map as map

pl.Config.set_tbl_rows(100)

In [ ]:
collision = fryer.data.uk_gov_dept_for_transport_road_accident.read_collision()
gdf = fryer.data.ons_local_authority_district_boundaries.read()

print(
    "collision",
    f"rows: {collision.shape[0]:,}",
    f"columns: {collision.shape[1]}",
    sep="\n",
)

In [ ]:
collisions_count = (
    collision.filter(pl.col("local_authority_ons_district").is_not_null())
    .group_by("local_authority_ons_district")
    .agg(
        [
            pl.col("accident_index").count().alias("accidents"),
            pl.col("number_of_casualties").sum().alias("casualties"),
            pl.col("number_of_vehicles").sum().alias("vehicles"),
        ]
    )
)

gdf2 = gdf.merge(
    collisions_count.to_pandas(),
    left_on="LAD24CD",
    right_on="local_authority_ons_district",
    how="left",
)

gdf2["geometry"] = gpd.GeoSeries(gdf2["geometry"]).simplify(tolerance=0.001)


uk_map = map.create()
colours = ["Reds", "Blues", "Greens"]

for index, col in enumerate(["accidents", "casualties", "vehicles"]):
    cp = folium.Choropleth(
        geo_data=gdf2,
        name=col,
        data=gdf2,
        columns=["LAD24CD", col],
        key_on="feature.properties.LAD24CD",
        fill_color=colours[index],
        bins=6,
        fill_opacity=0.6,
        line_opacity=0.4,
        smooth_factor=0,
        # use_jenks=True,
        overlay=True,
        highlight=True,
    ).add_to(uk_map)

    folium.GeoJsonTooltip(["LAD24NM", "LAD24CD", col], localize=True).add_to(cp.geojson)

folium.LayerControl().add_to(uk_map)

In [ ]:
uk_map

In [ ]:
# uk_map.save("uk_map.html")